In [ ]:
#Inspiration: https://analyticsindiamag.com/the-continuous-bag-of-words-cbow-model-in-nlp-hands-on-implementation-with-codes/

In [1]:
import numpy as np
import keras.backend as K
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim

In [2]:
data=pd.read_csv('/content/drive/MyDrive/DS-Data/amazon_alexa.tsv',sep='\t')
data

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
...,...,...,...,...,...
3145,5,30-Jul-18,Black Dot,"Perfect for kids, adults and everyone in betwe...",1
3146,5,30-Jul-18,Black Dot,"Listening to music, searching locations, check...",1
3147,5,30-Jul-18,Black Dot,"I do love these things, i have them running my...",1
3148,5,30-Jul-18,White Dot,Only complaint I have is that the sound qualit...,1


In [3]:
vectorize = Tokenizer()
vectorize.fit_on_texts(data.verified_reviews)
data= vectorize.texts_to_sequences(data.verified_reviews)
total_vocab = sum(len(s) for s in data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [15]:
data[0:2]

[[10, 7, 11], [267, 4]]

In [16]:
for s in data:
  print (s,len(s))
  break

[10, 7, 11] 3


In [ ]:
vectorize.word_index

In [10]:
word_count, total_vocab

(4255, 80200)

In [44]:
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    print(total_length)
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []            
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            contextual = sequence.pad_sequences(context_word, maxlen=total_length)
            final_target = np_utils.to_categorical(target, total_vocab)
            print(begin,end,idx,word,contextual, final_target, final_target.shape)
            if idx==2: break

In [ ]:
help(np_utils.to_categorical)

In [ ]:
help(tf.keras.backend.categorical_crossentropy)

In [ ]:
import tensorflow as tf
a = tf.keras.utils.to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)

b = tf.constant([.9, .04, .03, .03,
                      .3, .45, .15, .13,
                      .04, .01, .94, .05,
                      .12, .21, .5, .17],
                    shape=[4, 4])
loss = tf.keras.backend.categorical_crossentropy(a, b)
print (loss)

In [45]:
cbow_model(data, window_size, total_vocab)

Streaming output truncated to the last 5000 lines.
-2 3 0 67 [[ 0  0 24 27]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-1 4 1 24 [[ 0 67 27 47]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
0 5 2 27 [[67 24 47 27]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-2 3 0 103 [[  0   0 487  12]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-1 4 1 487 [[  0 103  12  27]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
0 5 2 12 [[103 487  27   6]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-2 3 0 10 [[0 0 0 4]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-1 4 1 4 [[ 0  0  0 10]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-2 3 0 87 [[  0   0 247  51]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-1 4 1 247 [[ 0 87 51 11]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
0 5 2 51 [[ 87 247  11 136]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-2 3 0 1034 [[ 0  0 21  7]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-1 4 1 21 [[   0 1034    7  253]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
0 5 2 7 [[1034   21  253 2284]] [[0. 0. 0. ... 0. 0. 0.]] (1, 80200)
-2 3 0 349 [[   0    0 1410    5]] [

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for contextual, final_target in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(contextual, final_target)
        print (cost)
    print(i, cost)

In [ ]:
dimensions=100
vect_file = open('/content/drive/MyDrive/DS-Data/vectors.txt' ,'w')
vect_file.write('{} {}\n'.format(total_vocab,dimensions))

In [ ]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [ ]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/DS-Data/vectors.txt', binary=False)
cbow_output.most_similar(positive=['virus'])